In [1]:
import torch
import torch.nn as nn
import os

class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, hidden_dim, layers):
        super(NeuMF, self).__init__()
        self.gmf_user_emb = nn.Embedding(num_users, hidden_dim)
        self.gmf_item_emb = nn.Embedding(num_items, hidden_dim)

        self.mlp_user_emb = nn.Embedding(num_users, hidden_dim)
        self.mlp_item_emb = nn.Embedding(num_items, hidden_dim)

        layers = [layers] if type(layers) is int else layers
        layers = [2*hidden_dim] + layers
        self.mlp_fc_layers = nn.ModuleList(nn.Linear(layers[i], layers[i+1]) for i in range(len(layers) -1))

        self.last = nn.Linear(hidden_dim + layers[-1], 1)
    
    def forward(self, user_indices, item_indices):
        gmf_u = self.gmf_user_emb(user_indices)
        gmf_i = self.gmf_item_emb(item_indices)
        gmf_layer= torch.mul(gmf_u, gmf_i)

        mlp_u = self.mlp_user_emb(user_indices)
        mlp_i = self.mlp_item_emb(item_indices)
        mlp_layer = torch.cat([mlp_u, mlp_i], dim = -1)
        for layer in self.mlp_fc_layers:
            mlp_layer = layer(mlp_layer)
            mlp_layer = torch.relu(mlp_layer)

        nmf_layer = torch.cat([gmf_layer, mlp_layer], dim = -1)
        output = torch.sigmoid(self.last(nmf_layer))

        return output

    def load_pretrained_weights(self, path='./checkpoint'):
        gmf_path = os.path.join(path, 'gmf.pt')
        mlp_path = os.path.join(path, 'mlp.pt')

        assert os.path.isfile(gmf_path) and os.path.isfile(mlp_path)
        self.load_state_dict(torch.load(gmf_path), strict=False)
        self.load_state_dict(torch.load(mlp_path), strict=False)

In [2]:
import pandas as pd
from utils.data import SampleGenerator

ratings = pd.read_csv('dataset/ratings.csv')
ratings = ratings.rename(columns={'movieId': 'itemId'})

userId = list(set(ratings.userId))
new_userId = list(range(0,len(userId)))

df = pd.DataFrame({'userId':userId,'new_userId':new_userId})
ratings = pd.merge(ratings,df,how='left', on='userId')

itemId = list(set(ratings.itemId))
new_itemId = list(range(0,len(itemId)))

df = pd.DataFrame({'itemId':itemId,'new_itemId':new_itemId})
ratings = pd.merge(ratings,df,how='left', on='itemId')

ratings = ratings.drop(['userId', 'itemId'],axis = 1)
ratings = ratings.rename(columns={'new_userId':'userId', 'new_itemId':'itemId'})

data = SampleGenerator(ratings, implicit=True)
hidden_dim = 128
layers = [128]
lr = 0.0001
batch_size = 2048
epochs = 15

use_pretrained_weights = True

num_users = data.num_users
num_items = data.num_items
num_negatives_train = 5
num_negatives_test = 500

cuda =  torch.cuda.is_available()

model = NeuMF(num_users, num_items, hidden_dim,layers)
criterion = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr)

if cuda:
    model.cuda()

if use_pretrained_weights:
    model.load_pretrained_weights('./checkpoint')

C:\Users\hojin\Desktop\code\recsys-implementation-pytorch\1. Neural Collaborative Filtering\utils\data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ratings['rating'][ratings['rating'] >0] = 1.0


In [3]:
import os
from utils.eval import Evaluation

test_loader, negative_loader = data.instance_test_loader(num_negatives = num_negatives_test, batch_size = batch_size)
  
for epoch in range(1,epochs+1):

    train_loader = data.instance_a_train_loader(num_negatives=num_negatives_train, batch_size=batch_size)
    total_loss = 0
    for batch_id, batch in enumerate(train_loader):
        user, item, rating = batch[0], batch[1], batch[2]
        rating = rating.float()
        if cuda:
            user, item, rating = user.cuda(), item.cuda(), rating.cuda()
        optim.zero_grad()
        pred = model(user,item)
        loss = criterion(pred.view(-1), rating)
        loss.backward()
        optim.step()
        total_loss += loss.item()
    print("epoch{0} loss:{1:.4f}".format(epoch, total_loss))
    
    with torch.no_grad():
        test_users, test_items, test_preds = list(), list(), list()
        neg_users, neg_items, neg_preds = list(), list(), list()

        for batch in test_loader:
            user, item = batch[0], batch[1]
            test_users += user.data.view(-1).tolist()
            test_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred = pred.cpu()

            test_preds += pred.data.view(-1).tolist()

        for batch in negative_loader:
            user, item = batch[0], batch[1]
            neg_users += user.data.view(-1).tolist()
            neg_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred =  pred.cpu()

            neg_preds += pred.data.view(-1).tolist()

        eval = Evaluation([test_users, test_items, test_preds,
                           neg_users, neg_items, neg_preds])
        eval.print_eval_score_k(10)

epoch1 loss:108.1789
recall@10:0.5337, prec@10:0.2250
epoch2 loss:64.4842
recall@10:0.6757, prec@10:0.3128
epoch3 loss:53.3960
recall@10:0.7216, prec@10:0.3471
epoch4 loss:47.8840
recall@10:0.7448, prec@10:0.3664
epoch5 loss:44.5162
recall@10:0.7572, prec@10:0.3767
epoch6 loss:42.4174
recall@10:0.7657, prec@10:0.3834
epoch7 loss:40.9052
recall@10:0.7710, prec@10:0.3885
epoch8 loss:39.8620
recall@10:0.7736, prec@10:0.3906
epoch9 loss:39.0525
recall@10:0.7782, prec@10:0.3939
epoch10 loss:38.4734
recall@10:0.7797, prec@10:0.3946
epoch11 loss:37.9541
recall@10:0.7810, prec@10:0.3959
epoch12 loss:37.4141
recall@10:0.7833, prec@10:0.3981
epoch13 loss:37.0464
recall@10:0.7843, prec@10:0.3987
epoch14 loss:36.7406
recall@10:0.7851, prec@10:0.3992
epoch15 loss:36.2540
recall@10:0.7861, prec@10:0.3995
